# Importing libraries

In [29]:
from police_api import PoliceAPI
import pandas as pd
import re
import numpy as np

# Defining main variables

We assign all england forces to a variable and define a london force by selecting london force from all england forces.

In [30]:
api = PoliceAPI()

In [31]:
all_forces = api.get_forces()    

In [32]:
london_force = all_forces[24] 

In [33]:
london_nei = london_force.neighbourhoods     #assigning neighbourhoods in london in variable

# Creating neighbourhood code and name dataframe


In [34]:
df_code = pd.DataFrame({'code': london_nei})       #creating dataframe with neighbourhood code
df_code.head()

,code
0,<Neighbourhood> E05000026
1,<Neighbourhood> E05000455
2,<Neighbourhood> E05000630
3,<Neighbourhood> E05000214
4,<Neighbourhood> E05009388


In [35]:
df_code['Neighbourhood'] = df_code.apply(lambda x: x['code'].name, axis=1)

#assigning neighbourhood names to the neighbourhood code and adding a new column with them to the dataframe

df_code.head()

,code,Neighbourhood
0,<Neighbourhood> E05000026,Abbey
1,<Neighbourhood> E05000455,Abbey
2,<Neighbourhood> E05000630,Abbey Road
3,<Neighbourhood> E05000214,Abbey Wood
4,<Neighbourhood> E05009388,Abingdon


In [36]:
df_code.code=df_code.code.astype(str)  
df_code['code']=df_code['code'].apply(lambda x:x.split(">")[1])
df_code #cleaning NH column, leaving the digit code

,code,Neighbourhood
0,E05000026,Abbey
1,E05000455,Abbey
2,E05000630,Abbey Road
3,E05000214,Abbey Wood
4,E05009388,Abingdon
...,...,...
653,E05000230,Woolwich Riverside
654,E05000572,Worcester Park
655,E05000265,Wormholt and White City
656,E05000344,Yeading


# Main loop to import data from api

In [17]:
crimes=[]
nei_crimes=[]
officers=[]
nei_officers=[]
longitudes=[]
latitudes=[]
for i in london_nei:
    for z in i.officers:
        nei_officers.append(i.name)
        officers.append(z)   
    for j in i.crimes:       
        crimes.append(i.name)
        new_crimes.append(j.category)
        longitudes.append(j.location.longitude)
        latitudes.append(j.location.latitude)

APIError: <unknown error code>

# Creating dataframe with crimes and their location 

In [125]:
df_crimes=pd.DataFrame(nei_crimes)  
df_crimes['crimes']=crimes
df_crimes['latitude']=latitudes
df_crimes['longitude']=longitudes
df_crimes

,0,crimes,latitude,longitude
0,Abbey,<CrimeCategory> Anti-social behaviour,51.538296,0.087049
1,Abbey,<CrimeCategory> Anti-social behaviour,51.538296,0.087049
2,Abbey,<CrimeCategory> Anti-social behaviour,51.542615,0.081852
3,Abbey,<CrimeCategory> Anti-social behaviour,51.540150,0.082375
4,Abbey,<CrimeCategory> Anti-social behaviour,51.541146,0.082002
...,...,...,...,...
85864,Yiewsley,<CrimeCategory> Violence and sexual offences,51.511575,-0.465497
85865,Yiewsley,<CrimeCategory> Violence and sexual offences,51.520429,-0.463844
85866,Yiewsley,<CrimeCategory> Violence and sexual offences,51.514661,-0.466993
85867,Yiewsley,<CrimeCategory> Other crime,51.509944,-0.474198


# New dataframe with neighbourhoods and their officers

In [126]:
df_officers=pd.DataFrame(nei_officers)  
df_officers['officers']=officers

In [28]:
df_officers.officers=df_officers.officers.astype(str)  #cleaning officers column name
df_officers['officers']=df_officers['officers'].apply(lambda x:x.split(">")[1])
df_officers.rename(columns={0:'Neighbourhood'},inplace=True)

NameError: name 'df_officers' is not defined

In [127]:
df_crimes.crimes=df_crimes.crimes.astype(str)  #cleaning in crimes dataframe
df_crimes['crimes']=df_crimes['crimes'].apply(lambda x:x.split(">")[1])
df_crimes

# Removing anti-social behaviour from crime dataframe 

Importing crime dataframe with location to a csv. in order to create a map with tableau.

In [145]:
not_antisocial=df_crimes.loc[df_crimes['crimes']!=' Anti-social behaviour']
not_antisocial
not_antisocial.to_csv('/Users/islaam/Desktop/anti.csv')

# Renaming neighbourhood column in dataframes

In [129]:
df_officers.rename(columns={0:'Neighbourhood'},inplace=True) 

In [130]:
df_crimes.rename(columns={0:'Neighbourhood'},inplace=True)

# Grouping crimes by neighbourhood 

In [131]:
crime_type_nh = df_code.merge(df_crimes, how='right').groupby(['Neighbourhood','crimes']).count()
crime_type_nh.drop(columns=['latitude','longitude'],inplace=True) 
crime_type_nh.rename(columns={'code':'total crimes'},inplace=True)
crime_type_nh

total crimes
Neighbourhood crimes                                     
Abbey          Anti-social behaviour                  154
               Bicycle theft                            8
               Burglary                                28
               Criminal damage and arson               36
               Drugs                                   40
...                                                   ...
Yiewsley       Robbery                                  5
               Shoplifting                             12
               Theft from the person                    2
               Vehicle crime                           16
               Violence and sexual offences            33

[7626 rows x 1 columns]

In [132]:
crime_nh=df_crimes.groupby(['Neighbourhood']).count()
crime_nh.drop(columns=['latitude','longitude'],inplace=True) 
crime_nh = crimepernh.sort_values('crimes',ascending=False)
crime_nh.reset_index(inplace=True)

# Grouping officers per neighbourhood

In [133]:
officers_nh = df_officers.groupby(['Neighbourhood']).count()
officers_nh = officers_nh.sort_values('officers',ascending=False)
officers_nh.reset_index(inplace=True)

,crimes,officers
Neighbourhood,,
Abbey,351.0,15
Abbey Road,72.0,5
Abbey Wood,185.0,4
Abingdon,108.0,5
Acton Central,141.0,5
...,...,...
Woolwich Riverside,342.0,6
Worcester Park,54.0,4
Wormholt and White City,102.0,3


# Merging total crimes per neighbourhood with officers

In [ ]:
crimes_officers = crime_nh.merge(officers_nh, how='outer').groupby(['Neighbourhood']).sum()
crimes_officers

# Counting total crimes in London per category

In [137]:
crimes_london=df_crimes.groupby(['crimes']).count()
crimes_london.drop(columns=['latitude','longitude'],inplace=True) 
crimes_london.rename(columns={'Neighbourhood':'total crimes'},inplace=True)
crimes_london.sort_values('total crimes',ascending=False,inplace=True)
crimes_london


,total crimes
crimes,
Anti-social behaviour,19866
Violence and sexual offences,18499
Vehicle crime,8974
Other theft,8166
Burglary,6198
Public order,4047
Criminal damage and arson,3975
Drugs,3646
Shoplifting,3314
